In [ ]:
'''
O que é Subword Tokenization?
É um meio-termo entre tokenizar por palavras e por caracteres.

🔎 Por quê? 👉 Tokenizar só por palavras quebra com palavras raras, nomes próprios ou erros de digitação.
👉 Tokenizar só por caracteres perde contexto e gera sequências gigantes.

Subword Tokenization resolve isso quebrando palavras em pedaços menores, mas inteligentes.

---------- ---------- ----------


'''

In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokens = tokenizer.tokenize("I love programação")
print(tokens)
# ['i', 'love', 'program', '##a', '##ção']


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

['i', 'love', 'program', '##aca', '##o']


In [ ]:
'''
O que é o WordPiece?
👉 Algoritmo de tokenização por subwords criado pelo Google
👉 Usado no BERT e derivados
👉 Baseia-se na frequência de pares de tokens para gerar um vocabulário eficiente

Resumo:

Começa quebrando tudo em caracteres

Vai juntando os pares mais frequentes

Cria um vocabulário balanceado entre tamanho e cobertura
'''

In [5]:
# Lista de "palavras" de treinamento (vocabulário inicial)
corpus = ["eu gosto de programar", "gosto de programacao", "programar e divertido"]

# Contador para pares de símbolos
from collections import Counter, defaultdict

# Função que realiza o treinamento do WordPiece
def train_wordpiece(corpus, vocab_size=20):
    # Passo 1: Quebra cada palavra em caracteres e adiciona o marcador de fim de palavra '_'
    # Ex: "programar" -> ['p', 'r', 'o', 'g', 'r', 'a', 'm', 'a', 'r', '_']
    corpus_tokens = []
    for sentence in corpus:
        for word in sentence.strip().split():
            corpus_tokens.append(list(word) + ['_'])  # O '_' ajuda a indicar fim de palavra

    # Passo 2: Cria o vocabulário inicial com todos os caracteres + '_'
    vocab = set()
    for token in corpus_tokens:
        vocab.update(token)

    # Passo 3: Constrói pares de símbolos mais frequentes (bigramas) e treina até chegar no vocab_size
    while len(vocab) < vocab_size:
        # Conta todos os pares adjacentes de caracteres (ou subwords) no corpus
        pairs = Counter()
        for token in corpus_tokens:
            for i in range(len(token) - 1):
                pairs[(token[i], token[i + 1])] += 1

        # Se não houver mais pares pra combinar, termina o treinamento
        if not pairs:
            break

        # Escolhe o par mais frequente
        most_common = pairs.most_common(1)[0][0]

        # Faz o merge (união) desse par mais comum em todo o corpus
        new_corpus_tokens = []
        for token in corpus_tokens:
            new_token = []
            i = 0
            while i < len(token):
                # Se o par encontrado está na posição atual, faz o merge
                if i < len(token) - 1 and (token[i], token[i + 1]) == most_common:
                    new_token.append(token[i] + token[i + 1])  # Une o par
                    i += 2  # Pula o próximo porque já foi unido
                else:
                    new_token.append(token[i])  # Mantém o símbolo atual
                    i += 1
            new_corpus_tokens.append(new_token)

        corpus_tokens = new_corpus_tokens

        # Atualiza o vocabulário com o novo token criado
        vocab.add(most_common[0] + most_common[1])

    # Retorna o vocabulário final treinado
    return vocab

# Treinando o WordPiece simples
vocab_final = train_wordpiece(corpus, vocab_size=20)

# Exibe o vocabulário aprendido
print("Vocabulário treinado:", vocab_final)


Vocabulário treinado: {'c', 'i', 'p', 'e_', 'u', 'pr', 't', 'prog', 'v', 'g', 'o', 'm', 'a', '_', 'e', 'r', 's', 'd', 'o_', 'pro'}


In [ ]:
'''
O que é o SentencePiece?
✅ Trabalha direto nos caracteres (byte-level)
✅ Trate todo o texto como uma sequência contínua ("raw text")
✅ Lida melhor com idiomas que não têm separação clara de palavras (japonês, chinês, coreano)
✅ Produz subwords como o BPE, mas pode gerar tokens começando com o símbolo especial ▁ (underline invisível) que representa espaço.
'''

In [6]:
!pip install sentencepiece

In [12]:
import sentencepiece as spm

# Passo 1 - Cria um arquivo de texto simulando o corpus
with open('corpus.txt', 'w', encoding='utf-8') as f:
    f.write("Eu gosto de programar computadores.\n")
    f.write("Programação é divertida.\n")
    f.write("Eu amo programar.\n")
    f.write("Computadores são incríveis.\n")

# Passo 2 - Treinando o modelo SentencePiece
spm.SentencePieceTrainer.train(
    input='corpus.txt',          # Corpus de treino
    model_prefix='mymodel',      # Nome dos arquivos de saída (mymodel.model e mymodel.vocab)
    vocab_size=50,               # Tamanho do vocabulário
    model_type='bpe',            # Pode ser 'bpe', 'unigram', 'char', 'word' -- bpe = byte pair encoding
    pad_id=0,                    # ID do padding
    unk_id=1,                    # ID para palavras desconhecidas
    bos_id=2,                    # ID do token <BOS>
    eos_id=3,                    # ID do token <EOS>
    user_defined_symbols=['<sep>', '<cls>']  # Exemplo de tokens personalizados
)

# Passo 3 - Carrega o modelo treinado
sp = spm.SentencePieceProcessor()
sp.load('mymodel.model')

# Passo 4 - Testa a tokenização de uma frase nova
text = "Eu adoro programar novos computadores"
tokens = sp.encode(text, out_type=str)  # Retorna as subwords como strings
print("Tokens:", tokens)

# Passo 5 - Decodificando os tokens de volta para a frase original
decoded = sp.decode(tokens)
print("Frase reconstruída:", decoded)


Tokens: ['▁Eu', '▁', 'ad', 'o', 'ro', '▁p', 'rograma', 'r', '▁', 'n', 'o', 'v', 'o', 's', '▁', 'c', 'omput', 'ador', 'es']
Frase reconstruída: Eu adoro programar novos computadores


In [8]:
'''
Visualizar o arquivo .vocab
'''

with open('mymodel.vocab', 'r', encoding='utf-8') as f:
    for i in range(10):  # Exibe as 10 primeiras linhas
        print(f.readline().strip())

'''
poderia ser usado comando: `more mymodel.vocab` num cmd do windows para ver o arquivo, mas estou usando collab do google :D
linux: `head mymodel.vocab`
'''

<pad>	0
<unk>	0
<s>	0
</s>	0
<sep>	0
<cls>	0
am	-0
gr	-1
ro	-2
ama	-3


In [ ]:
'''
Como plugar o modelo .model do SentencePiece direto num tokenizer da Hugging Face.

Resumo do fluxo
Treina o mymodel.model com SentencePiece ✅
Usa o AutoTokenizer ou PreTrainedTokenizerFast do Hugging Face ✅
Pluga o .model direto ✅
Tokeniza textos ✅
'''

In [14]:
import os
print(os.path.abspath("./mymodel.model"))

/content/mymodel.model


In [15]:
from transformers import PreTrainedTokenizerFast

# Carrega o tokenizer com o SentencePiece model
tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="mymodel.model",
    unk_token="<unk>",
    pad_token="<pad>",
    bos_token="<s>",
    eos_token="</s>"
)

# Testa tokenizando um texto
text = "Eu adoro programar novos computadores"
tokens = tokenizer(text)
print("Tokens:", tokens)

# Decodifica de volta
decoded = tokenizer.decode(tokens['input_ids'])
print("Frase reconstruída:", decoded)


Exception: stream did not contain valid UTF-8